In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
from IPython.display import display
import seaborn as sns

## Filter data

In [2]:
def filter_data(data_sent , max_flux= -12):
    data = data_sent.copy()
    max_flux = -12
    min_flux = 26
    data = data[data['flux_aper']<max_flux]

    data = data[data['significance']>2]
    data_class = data[['class']]

    data_sig = data['significance']
    data_id = data['src_id']
    data_name = data['src_n']
    obs_info_params = [ 'livetime','likelihood','pileup_flag','mstr_sat_src_flag','mstr_streak_src_flag'   ,'gti_obs' , 'flux_significance_b'  , 'flux_significance_m' , 'flux_significance_s' , 'flux_significance_h' , 'flux_significance_u'    ]
    data_val = data.drop(columns=obs_info_params)
    return data_val

def norm_data(data_sent):
    data = data_sent.copy()
    #data.replace()
    for d in data:
        max_val = np.amax(data[d])
        min_val =  np.amin(data[d])
        data[d] = (data[d]-min_val)/(max_val-min_val)
    return data
def std_data(data_sent):
    data = data_sent.copy()
    for d in data:
        mean =  np.mean(data[d])
        std = np.sqrt(np.var(data[d]))
        data[d] = (data[d]-mean)/std 
    return data
def do_nothing(data_sent):
    return data_sent

In [3]:
def extract_data(data_sent , impute_fn = '',reduce_fn = ' ' , rf_impute=False):
    data = data_sent.copy()
    data = data.sample(frac=1)
    data = filter_data(data)
    #display(data)
    data_id = data[[ 'class' ,'src_n' , 'src_id' ,'significance' , ]]
    data_id = data_id
    data_val = data.drop([ 'class' ,'src_n' , 'src_id' ,'significance' ,] , axis=1)
    data_val = reduce_fn(data_val)
    return data_val , data_id
    #if(rf_impute):
    #    data_val  , random_forest_imputer = impute_fn(data_val , data_id)
    #else:
    #    data_val = impute_fn(data_val)
    data_val = reduce_fn(data_val)
    data_val = data_val.reset_index(drop=True)
    data_reduced = pd.concat([data_id , data_val] , axis=1)
    if(rf_impute):
        return(data_reduced , random_forest_imputer)
    else:
        return data_reduced

## Load data

load all data

### All data imputation at once

In [4]:

train = pd.read_csv('../processed_data/'+'BH'+'_.csv' , index_col='obs_id')
df_bh = train.sample(frac=1)


train = pd.read_csv('../processed_data/'+'NS'+'_.csv' , index_col='obs_id')
df_ns = train.sample(frac=1)

train = pd.read_csv('../processed_data/'+'CV'+'_.csv' , index_col='obs_id')
df_cv = train.sample(frac=1)

train = pd.read_csv('../processed_data/'+'PULSAR'+'_.csv' , index_col='obs_id')
df_plsr = train.sample(frac=1)

train = pd.read_csv('../processed_data/'+'TUC'+'_.csv' , index_col='obs_id')
df_tuc = train.sample(frac=1)
src = pd.read_csv('../source_list/TUC_data_clean_source_list.csv')
#display(src.head())
tuc_class = ['CV           ','MSP          ', 'QLX          ']
src_list = src[src['A_SOURCE_TYPE'].isin(tuc_class)]['SRC_ID']
df_tuc = df_tuc[df_tuc['src_id'].isin(src_list)]

train = pd.concat([df_bh , df_ns , df_cv, df_plsr , df_tuc])
#train

Le us see number of sources in each class 

In [5]:
num_src = len(np.unique(train['src_id']))
print('Total Num of sources : ' , num_src)

Total Num of sources :  337


In [7]:
obs_count = train['src_id'].value_counts().to_frame(name = 'num_obs')
obs_count[obs_count['num_obs']==1]

,num_obs
PL0022,1
PL0019,1
PL0092,1
PL0070,1
PL0045,1
...,...
PL0073,1
BH0029,1
PL0061,1
PL0054,1


### Select only TUC sources for which classification is given

In [8]:
train['class'].value_counts()

CV        786
TUC       656
PULSAR    445
NS        440
BH        199
Name: class, dtype: int64

# RF imputer Missingpy

In [9]:
import sklearn.neighbors._base
from os import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [10]:
from missingpy import MissForest 

In [11]:
def rf_impute(d, i ):
    data = pd.concat([i , d] , axis=1)
    data = data.drop(columns=['src_n' , 'src_id' , 'significance' ,])
    rf_imputer = MissForest(verbose=0 , decreasing=True)
    #new_data = d.drop(columns= ['class'])
    new_data = rf_imputer.fit_transform(d)
    return new_data , rf_imputer

# Choose Features

In [12]:
%load_ext autoreload
%autoreload 2


In [13]:
from features import features as feat 
all_feat = list(feat['info'])+list(feat['flux']['photon'])+list(feat['flux']['energy'])+list(feat['variability'])+list(feat['hardness'])+list(feat['model_fit']['powerlaw'])+list(feat['model_fit']['bb'])+list(feat['model_fit']['brems'])+list(feat['info_pre_filter'])

In [14]:
feat_to_drop = list(feat['model_fit']['powerlaw']) + list(feat['model_fit']['bb']) + list(feat['model_fit']['brems']) 
#feat_to_drop = []
feat_used = [item for item in all_feat if item not in feat_to_drop]

In [15]:
sp = [] 
for f in feat_used:
    #print(f)
    na = train[feat_used][f].isna().value_counts()
    try:
        sp.append([f , 1-na[0]/(na[0]+na[1])])
    except:
        sp.append([f , 0])
sp =  np.asarray(sp)
sparsity = pd.DataFrame(sp , columns=['feat' , 'sparsity']).sort_values(by='sparsity' , ascending=False).reset_index()


In [17]:
u_band = ['flux_aper_lolim_u' , 'flux_aper_hilim_u' , 'flux_aper_u', 'photflux_aper_u' ,'photflux_aper_lolim_u' , 'photflux_aper_hilim_u' ]

In [18]:
top_sparse = sparsity[:4]['feat']
top_sparse

0        flux_aper_lolim_u
1              flux_aper_u
2    photflux_aper_lolim_u
3          photflux_aper_u
Name: feat, dtype: object

In [19]:
train_set = train[feat_used]
train_set = train_set.drop(columns=u_band)


In [20]:
from tqdm import tqdm

In [28]:
cl = str(input('Enter class :'))
source_list = train_set[train_set['class']==cl]['src_n'].to_list()
all_imp_data = pd.DataFrame()
all_imp_id = pd.DataFrame()
all_imp_data_norm = pd.DataFrame()
s  = source_list[8]
for s in tqdm(source_list[:1]):
    s = 'CXOU J100510.9-07453'
    #print(s)
    train_now = train_set[train_set['src_n']==s]
    data_val , data_id   = extract_data(train_now ,  impute_fn= rf_impute , reduce_fn= do_nothing , rf_impute=True )
    print('Source :' ,  s)
    print('Num of obs : ' , len(data_val))
    try:
        new_data , random_forest_imputer = rf_impute(data_val, data_id)
        imp_data =  pd.DataFrame(new_data , columns = data_val.columns.to_list()  , index=data_val.index.to_list())
        imp_data.index.name = 'obs_id'
        normalized_df=(imp_data-imp_data.mean())/imp_data.std()
    except :
        print('all columns missing')
        imp_data = data_val.copy()
        imp_data.index.name = 'obs_id'
        normalized_df = data_val.copy()
    #display(imp_data)
    #normalized_df.describe()
    all_imp_data = all_imp_data.append(imp_data)
    all_imp_data_norm = all_imp_data_norm.append(normalized_df)
    all_imp_id = all_imp_id.append(data_id)

100%|██████████| 1/1 [00:00<00:00, 73.18it/s]

Source : CXOU J100510.9-07453
Num of obs :  5
all columns missing


In [47]:
len(all_imp_id)

16

In [51]:

final_data =  pd.concat([all_imp_id , all_imp_data] , axis=1)
final_data_norm = pd.concat([all_imp_id , all_imp_data_norm] , axis=1)
display(final_data)
display(final_data_norm)

,class,src_n,src_id,significance,photflux_aper_hilim_h,photflux_aper_b,photflux_aper_hilim_b,photflux_aper_hilim_s,photflux_aper_hilim_m,photflux_aper_m,...,ks_prob,hard_ms_hilim,hard_hm_lolim,hard_hs_hilim,hard_ms_lolim,hard_hm_hilim,hard_hs_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
BH_OBS_77,BH,CXOU J100515.4-07425,BH0019,17.39,-6.445148,-5.854182,-5.801618,-6.151996,-6.461426,-6.552532,...,0.399000,-0.2280,-0.1843,-0.2117,-0.4878,0.1818,-0.5103,0.0037,-0.3635,-0.3623
BH_OBS_79,BH,CXOU J100515.4-07425,BH0019,17.39,-6.522734,-5.790753,-5.731890,-6.028956,-6.393189,-6.484656,...,0.347500,-0.2879,-0.3879,-0.4179,-0.5403,0.0244,-0.6977,-0.1811,-0.5646,-0.4197
BH_OBS_78,BH,CXOU J100515.4-07425,BH0019,17.39,-7.483729,-6.606776,-6.479779,-6.739690,-7.003839,-7.176200,...,0.781500,-0.0369,-1.0000,-0.5890,-0.6040,-0.3429,-1.0000,-0.9994,-0.9994,-0.3410
BH_OBS_76,BH,CXOU J100515.4-07425,BH0019,17.39,-6.574955,-5.842543,-5.793174,-6.198185,-6.347754,-6.419531,...,0.704000,-0.0394,-0.4541,-0.2979,-0.2929,-0.1343,-0.5828,-0.2948,-0.4447,-0.1699
BH_OBS_75,BH,CXOU J100515.4-07425,BH0019,17.39,-6.731890,-6.374071,-6.242832,-6.566550,-6.926282,-7.156830,...,0.006000,-0.1205,-0.1468,0.1405,-0.7639,0.6552,-0.5778,0.2848,-0.2299,-0.4735
BH_OBS_71,BH,CXOU J100515.4-07425,BH0019,17.39,-6.427012,-6.174185,-6.021226,-6.503624,-6.760701,-7.011887,...,0.585000,0.0581,0.0406,0.4966,-0.7189,0.7876,-0.3054,0.4647,0.1174,-0.3635
BH_OBS_74,BH,CXOU J100515.4-07425,BH0019,17.39,-6.951170,-6.504872,-6.396531,-6.637895,-7.286090,-7.516555,...,0.557000,-0.4953,0.0893,-0.1205,-0.8826,0.7626,-0.6440,0.4622,-0.3948,-0.7108
BH_OBS_72,BH,CXOU J100515.4-07425,BH0019,17.39,-6.400226,-5.793984,-5.699622,-6.169155,-6.218747,-6.345823,...,0.378333,0.2067,-0.5590,0.0056,-0.3067,0.0493,-0.6115,-0.2523,-0.3123,-0.0562
BH_OBS_81,BH,CXOU J100515.4-07425,BH0019,17.39,-6.432151,-5.728158,-5.643783,-6.137570,-6.095393,-6.197363,...,0.279000,0.2942,-0.6652,-0.0981,-0.1455,-0.1930,-0.6427,-0.4372,-0.3848,0.0700


,class,src_n,src_id,significance,photflux_aper_hilim_h,photflux_aper_b,photflux_aper_hilim_b,photflux_aper_hilim_s,photflux_aper_hilim_m,photflux_aper_m,...,ks_prob,hard_ms_hilim,hard_hm_lolim,hard_hs_hilim,hard_ms_lolim,hard_hm_hilim,hard_hs_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
BH_OBS_77,BH,CXOU J100515.4-07425,BH0019,17.39,0.658824,0.448662,0.366428,0.474096,0.179049,0.267809,...,-0.249268,-0.671709,0.692423,-0.037777,0.052371,0.206375,0.723800,0.431609,0.292325,-0.283103
BH_OBS_79,BH,CXOU J100515.4-07425,BH0019,17.39,0.438636,0.633642,0.586313,0.919331,0.349471,0.416303,...,-0.466625,-0.928044,0.144280,-0.615952,-0.167429,-0.126851,-0.216602,0.065151,-0.337204,-0.537554
BH_OBS_78,BH,CXOU J100515.4-07425,BH0019,17.39,-2.288649,-1.746144,-1.772140,-1.652556,-1.175622,-1.096608,...,1.365088,0.146083,-1.503647,-1.095708,-0.434120,-0.904448,-1.733588,-1.557540,-1.698316,-0.188682
BH_OBS_76,BH,CXOU J100515.4-07425,BH0019,17.39,0.290433,0.482605,0.393055,0.306954,0.462945,0.558779,...,1.037996,0.135384,-0.033947,-0.279477,0.868354,-0.462829,0.359984,-0.160317,0.038134,0.569793
BH_OBS_75,BH,CXOU J100515.4-07425,BH0019,17.39,-0.154946,-1.067500,-1.024932,-1.026026,-0.981924,-1.054231,...,-1.907939,-0.211674,0.793382,0.949775,-1.103569,1.208593,0.385075,0.989031,0.710551,-0.776045
BH_OBS_71,BH,CXOU J100515.4-07425,BH0019,17.39,0.710292,-0.484568,-0.326103,-0.798318,-0.568386,-0.737136,...,0.535753,0.552625,1.297910,1.948262,-0.915168,1.488893,1.752019,1.345773,1.797750,-0.288423
BH_OBS_74,BH,CXOU J100515.4-07425,BH0019,17.39,-0.777257,-1.448960,-1.509620,-1.284195,-1.880542,-1.841215,...,0.417577,-1.815590,1.429023,0.217943,-1.600527,1.435966,0.052873,1.340816,0.194343,-1.827980
BH_OBS_72,BH,CXOU J100515.4-07425,BH0019,17.39,0.786311,0.624219,0.688071,0.412003,0.785138,0.720032,...,-0.336492,1.188543,-0.316364,0.571522,0.810578,-0.074136,0.215963,-0.076039,0.452604,1.073818
BH_OBS_81,BH,CXOU J100515.4-07425,BH0019,17.39,0.695710,0.816188,0.864158,0.526296,1.093215,1.044823,...,-0.755732,1.562989,-0.602281,0.280752,1.485469,-0.587100,0.059397,-0.442696,0.225647,1.633254


## Lets see how well imputations is

In [4]:
bh_imp = pd.read_csv('PULSAR_src_imp.csv').drop('src_n' , axis=1)
bh_imp

,obs_id,class,src_id,significance,photflux_aper_m,photflux_aper_hilim_h,photflux_aper_lolim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_hilim_b,...,var_prob,hard_hs_hilim,hard_hs,hard_hs_lolim,hard_ms_hilim,hard_hm_hilim,hard_hm,hard_ms_lolim,hard_hm_lolim,hard_ms
0,PULSAR_OBS_394,PULSAR,PL0108,15.97,-5.534766,-5.372020,-5.933301,-5.605723,-4.965372,-4.923360,...,0.352250,0.5228,0.4110,0.2904,0.4316,0.2180,0.1124,0.1893,0.0006,0.3111
1,PULSAR_OBS_392,PULSAR,PL0108,15.97,-5.236572,-4.959398,-5.757459,-5.301986,-4.563043,-4.526659,...,0.081750,0.6977,0.5884,0.4866,0.5315,0.3417,0.2611,0.2480,0.1793,0.3835
2,PULSAR_OBS_393,PULSAR,PL0108,15.97,-5.479910,-5.317494,-5.951947,-5.535212,-4.921543,-4.887060,...,0.231500,0.5928,0.5072,0.4141,0.4991,0.2142,0.1299,0.3029,0.0431,0.4022
3,PULSAR_OBS_243,PULSAR,PL0081,107.91,-6.541351,-4.504178,-6.967784,-7.047346,-4.428291,-4.362510,...,0.466000,0.9775,0.9219,0.8501,-0.3841,1.0000,0.9994,-1.0000,0.9613,-0.9994
4,PULSAR_OBS_230,PULSAR,PL0081,107.91,-6.823619,-4.834756,-7.672624,-7.124591,-4.716021,-4.692290,...,0.051000,1.0000,0.9994,0.9938,1.0000,0.9875,0.9781,0.5528,0.9675,0.9994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5251,PULSAR_OBS_21,PULSAR,PL0006,179.61,-3.917933,-3.793444,-4.053744,-3.926282,-3.282912,-3.277778,...,0.159200,0.2842,0.2711,0.2567,0.1580,0.1455,0.1324,0.1305,0.1193,0.1437
5252,PULSAR_OBS_22,PULSAR,PL0006,179.61,-4.086769,-3.924453,-4.215454,-4.093073,-3.451242,-3.447332,...,0.155000,0.3179,0.3086,0.2979,0.1493,0.1868,0.1774,0.1280,0.1680,0.1387
5253,PULSAR_OBS_24,PULSAR,PL0006,179.61,-4.103198,-3.920096,-4.238824,-4.115261,-3.463315,-3.456056,...,0.090200,0.3429,0.3235,0.3042,0.1580,0.2130,0.1949,0.1168,0.1768,0.1374
5254,PULSAR_OBS_23,PULSAR,PL0006,179.61,-4.083178,-3.913284,-4.209996,-4.088842,-3.443939,-3.440572,...,0.051200,0.3242,0.3161,0.3067,0.1468,0.1955,0.1874,0.1280,0.1793,0.1374


In [55]:
np.unique(train_set['class'])

array(['BH', 'CV', 'NS', 'PULSAR', 'TUC'], dtype=object)

In [54]:
final_data.to_csv(cl+'_src_imp.csv')
final_data_norm.to_csv(cl+'_src_imp_norm.csv')

In [24]:
imp_data =  pd.DataFrame(new_data , columns = data_val.columns.to_list()  , index=data_val.index.to_list())
imp_data.index.name = 'obs_id'
normalized_df=(imp_data-imp_data.mean())/imp_data.std()
display(imp_data)
normalized_df.describe()

ValueError: Shape of passed values is (11, 50), indices imply (5, 50)

In [ ]:
data_id

,class,src_n,src_id,significance
obs_id,,,,
TUC_OBS_456,TUC,CXOGLB J002405.3-720421,TC0045,10.20
CV_OBS_707,CV,XMMSL2 J004231.9+41162,CV0110,174.76
NS_OBS_51,NS,1A 1744-361,NS0004,11.50
NS_OBS_401,NS,SAXWFC J1748.1-2446.,NS0072,19.47
PULSAR_OBS_118,PULSAR,PSR J0729-1448,PL0022,3.24
...,...,...,...,...
CV_OBS_393,CV,CXOGLB J002402.5-72051,CV0070,22.52
TUC_OBS_400,TUC,CXOGLB J002403.6-720459,TC0040,12.38
PULSAR_OBS_218,PULSAR,PSR J1744-1134,PL0080,14.80


In [ ]:
processed_data_all = pd.concat([data_id , normalized_df] , axis=1)
processed_data_all.to_csv('../processed_data/v3/'+'all_once'+'_rf_impute.csv')

processed_data_all = pd.concat([data_id , imp_data] , axis=1)
processed_data_all.to_csv('../processed_data/v3/'+'all_once'+'_rf_impute_no_norm.csv')

In [ ]:
processed_data_all = pd.concat([data_id , normalized_df] , axis=1)
processed_data_all.to_csv('../processed_data/v3/'+obj_class+'_rf_impute.csv')

In [ ]:
processed_data_all = pd.concat([data_id , imp_data] , axis=1)
processed_data_all.to_csv('../processed_data/v3/'+obj_class+'_rf_impute_no_norm.csv')

In [ ]:
processed_data_all

,class,src_n,src_id,significance,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
PULSAR_OBS_203,PULSAR,PSR J1714-3810,PL0073,29.01,-3.734946,-3.520281,-4.532392,-3.751291,-4.395558,-5.438422,...,0.738005,0.627700,0.674000,0.980000,0.960000,0.907600,0.817600,0.650800,0.969400,0.864500
PULSAR_OBS_247,PULSAR,PSR J1745-2900,PL0081,107.91,-4.556111,-4.568315,-4.670717,-4.627456,-6.318487,-5.690796,...,0.750000,0.960000,1.000000,0.975000,0.832600,-0.421600,-1.000000,0.999400,0.911900,-0.999400
PULSAR_OBS_259,PULSAR,PSR J1745-2900,PL0081,107.91,-4.758454,-4.707301,-5.180333,-4.800519,-6.814175,-6.726999,...,0.750000,0.981300,1.000000,1.000000,0.977500,0.258000,-1.000000,0.999400,0.999400,-0.451000
PULSAR_OBS_224,PULSAR,PSR J1745-2900,PL0081,107.91,-4.892790,-4.913996,-5.137785,-4.968592,-6.675512,-6.168386,...,0.750000,0.962500,1.000000,0.985000,0.875100,-0.325400,-1.000000,0.999400,0.938200,-0.999400
PULSAR_OBS_393,PULSAR,PSR J1856+0113,PL0108,15.97,-5.317494,-4.959398,-6.475055,-5.371509,-5.430860,-5.769040,...,0.828200,0.043100,0.214200,0.592800,0.414100,0.499100,0.302900,0.129900,0.507200,0.402200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PULSAR_OBS_183,PULSAR,PSR J1628-3205,PL0064,11.99,-5.060780,-4.629672,-5.407712,-5.116623,-5.272377,-5.019270,...,0.689148,0.154300,0.346700,0.051800,-0.143000,-0.200500,-0.385400,0.251100,-0.047500,-0.294800
PULSAR_OBS_242,PULSAR,PSR J1745-2900,PL0081,107.91,-4.658368,-4.695294,-4.684639,-4.738499,-6.320663,-5.707301,...,0.750000,0.950000,1.000000,0.968800,0.788900,-0.424100,-1.000000,0.999400,0.896900,-0.999400
PULSAR_OBS_20,PULSAR,PSR J0101-6422,PL0004,4.89,-5.781990,-4.979225,-5.071989,-5.987585,-5.437469,-4.934794,...,0.687307,-0.688900,-0.174300,-0.676500,-0.908800,-0.399100,-0.691400,-0.444700,-0.809500,-0.557200


In [ ]:
import joblib 
joblib.dump(random_forest_imputer , 'forest/v3/'+obj_class+'_imp.joblib' , compress=0)

['forest/v3/BH_imp.joblib']

# 47 TUC application

In [ ]:
trial = pd.read_csv('../processed_data/CV_.csv')
trial

,index,obs_id,class,src_n,src_id,livetime,significance,likelihood,pileup_flag,mstr_sat_src_flag,...,bb_ampl_lolim,bb_ampl_hilim,bb_stat,brems_kt,brems_kt_hilim,brems_kt_lolim,brems_nh,brems_nh_hilim,brems_nh_lolim,brems_stat
0,0,CV_OBS_0,CV,[DSH2013] 24,CV0001,9841.8,9.81,250.012,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,CV_OBS_1,CV,[DSH2013] 24,CV0001,66285.9,9.81,250.012,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,CV_OBS_2,CV,[DSH2013] 24,CV0001,52365.0,9.81,250.012,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,CV_OBS_3,CV,[HPH2013] 100,CV0002,4903.5,37.03,4392.422,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,CV_OBS_4,CV,[HPH2013] 100,CV0002,4108.6,37.03,4392.422,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,781,CV_OBS_781,CV,XMMSL2 J004231.9+41162,CV0110,3907.5,174.76,172643.300,False,False,...,0.000228,0.000440,2.256,4.334,6.475,3.119,5.096,13.680,NaN,0.465
782,782,CV_OBS_782,CV,XMMSL2 J004231.9+41162,CV0110,3907.5,174.76,172643.300,False,False,...,0.000159,0.000250,1.666,6.150,10.270,4.217,18.220,28.940,8.972,0.861
783,783,CV_OBS_783,CV,XMMSL2 J004231.9+41162,CV0110,3907.5,174.76,172643.300,False,False,...,0.000225,0.000392,2.444,6.490,12.520,4.064,5.549,14.910,NaN,0.942
784,784,CV_OBS_784,CV,XMMSL2 J004231.9+41162,CV0110,3974.2,174.76,172643.300,False,False,...,0.000348,0.000713,2.843,4.340,6.583,3.066,6.158,14.240,NaN,0.847


In [ ]:
tuc = pd.read_csv('../processed_data/TUC_.csv' ,index_col='obs_id')
tuc = tuc[feat_used]

In [ ]:
data_val , data_id   = extract_data(tuc,  impute_fn= rf_impute , reduce_fn= do_nothing , rf_impute=True )

In [ ]:
data_id

,class,src_n,src_id,significance
obs_id,,,,
TUC_OBS_1209,TUC,CXOGLB J002415.9-720451,TC0122,7.89
TUC_OBS_1065,TUC,CXOGLB J002407.8-720441,TC0108,24.52
TUC_OBS_1828,TUC,CXOGLB J002404.4-720432,TC0199,4.23
TUC_OBS_1401,TUC,CXOGLB J002353.4-720511,TC0148,5.20
TUC_OBS_2352,TUC,CXOGLB J002434.1-720333,TC0270,4.83
...,...,...,...,...
TUC_OBS_1547,TUC,CXOGLB J002359.3-720448,TC0168,3.42
TUC_OBS_976,TUC,CXOGLB J002406.3-720452,TC0101,7.20
TUC_OBS_1061,TUC,CXOGLB J002407.8-720441,TC0108,24.52


In [ ]:
data_val

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_1209,-6.995249,-6.667360,NaN,NaN,-7.145998,-6.187822,NaN,-6.294906,NaN,NaN,...,0.441000,-0.2367,1.0000,-0.6202,-1.0000,-0.7127,-1.0000,0.9981,-0.9994,-0.9994
TUC_OBS_1065,-5.852633,-5.183626,-5.766750,-5.926648,-5.672028,-5.651306,-5.788346,-5.142969,-5.726304,-5.942714,...,0.684400,-0.3304,-0.1131,-0.1355,-0.3567,0.0681,-0.1193,-0.2224,-0.2473,-0.0262
TUC_OBS_1828,NaN,NaN,NaN,NaN,-7.483861,-6.995679,NaN,NaN,NaN,NaN,...,0.222000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TUC_OBS_1401,-5.405718,-6.922269,NaN,-5.695725,-5.992679,-5.759201,NaN,-5.579879,NaN,NaN,...,0.489400,0.3816,1.0000,1.0000,0.1480,0.1268,-1.0000,0.9994,0.9994,-0.9994
TUC_OBS_2352,-6.668978,-6.898253,NaN,-6.927750,-6.844360,-7.109970,-7.764977,-6.565910,-7.096042,NaN,...,0.555500,-0.1755,0.6777,0.9225,0.1818,0.7951,-0.0993,0.2948,0.6408,0.4147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_1547,-5.852324,NaN,NaN,NaN,-6.229737,-6.053499,NaN,NaN,NaN,NaN,...,0.103250,0.0606,1.0000,1.0000,-0.1418,0.1718,-1.0000,0.9994,0.9994,-0.9994
TUC_OBS_976,-6.321937,-5.848630,-5.424350,-6.524329,-6.230918,-5.985900,-6.644357,-5.706859,-6.384787,-5.994391,...,0.583600,-0.4791,0.2030,-0.1330,-0.7039,-0.0269,-0.5528,-0.1387,-0.4397,-0.3086
TUC_OBS_1061,-5.945770,-5.403403,-5.691863,-6.078938,-5.903090,-5.717831,-6.153663,-5.318487,-6.007535,NaN,...,0.684400,-0.2929,0.1468,-0.0893,-0.4941,-0.0394,-0.4016,-0.0687,-0.2973,-0.2274


In [ ]:
tuc_imp =  random_forest_imputer.transform(data_val)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6


In [ ]:
imp_data =  pd.DataFrame(tuc_imp , columns = data_val.columns.to_list()  , index=data_val.index.to_list())
imp_data.index.name = 'obs_id'
normalized_df=(imp_data-imp_data.mean())/imp_data.std()
display(imp_data)
normalized_df.describe()

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_1209,-6.995249,-6.667360,-5.673662,-7.282439,-7.145998,-6.187822,-8.683497,-6.294906,-7.444246,-6.047718,...,0.441000,-0.236700,1.000000,-0.620200,-1.000000,-0.712700,-1.000000,0.998100,-0.999400,-0.99940
TUC_OBS_1065,-5.852633,-5.183626,-5.766750,-5.926648,-5.672028,-5.651306,-5.788346,-5.142969,-5.726304,-5.942714,...,0.684400,-0.330400,-0.113100,-0.135500,-0.356700,0.068100,-0.119300,-0.222400,-0.247300,-0.02620
TUC_OBS_1828,-7.222454,-8.289872,-6.767348,-7.258384,-7.483861,-6.995679,-8.583478,-7.180999,-7.500511,-7.961115,...,0.222000,-0.618144,0.497597,0.749703,-0.325799,0.414458,-0.677629,0.099233,0.229378,-0.14614
TUC_OBS_1401,-5.405718,-6.922269,-4.842619,-5.695725,-5.992679,-5.759201,-7.352701,-5.579879,-6.201074,-5.982962,...,0.489400,0.381600,1.000000,1.000000,0.148000,0.126800,-1.000000,0.999400,0.999400,-0.99940
TUC_OBS_2352,-6.668978,-6.898253,-6.699890,-6.927750,-6.844360,-7.109970,-7.764977,-6.565910,-7.096042,-7.894349,...,0.555500,-0.175500,0.677700,0.922500,0.181800,0.795100,-0.099300,0.294800,0.640800,0.41470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_1547,-5.852324,-6.898908,-5.115828,-6.112543,-6.229737,-6.053499,-7.505714,-5.985600,-6.409019,-6.063292,...,0.103250,0.060600,1.000000,1.000000,-0.141800,0.171800,-1.000000,0.999400,0.999400,-0.99940
TUC_OBS_976,-6.321937,-5.848630,-5.424350,-6.524329,-6.230918,-5.985900,-6.644357,-5.706859,-6.384787,-5.994391,...,0.583600,-0.479100,0.203000,-0.133000,-0.703900,-0.026900,-0.552800,-0.138700,-0.439700,-0.30860
TUC_OBS_1061,-5.945770,-5.403403,-5.691863,-6.078938,-5.903090,-5.717831,-6.153663,-5.318487,-6.007535,-6.113837,...,0.684400,-0.292900,0.146800,-0.089300,-0.494100,-0.039400,-0.401600,-0.068700,-0.297300,-0.22740


,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
count,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,...,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03
mean,-2.469272e-14,4.565192e-16,7.512227e-15,5.460150e-15,-1.565047e-14,2.635155e-15,3.948213e-15,-6.917848e-15,1.339613e-14,-3.377564e-15,...,-2.237396e-15,4.022793e-16,4.519992e-17,-2.033996e-17,-3.299594e-16,1.129998e-17,-1.083668e-15,1.807997e-16,-5.875990e-16,7.299787e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.406206e+00,-2.503640e+00,-1.141165e+00,-1.016288e+00,-1.723163e+00,-2.206655e+00,-1.422608e+00,-1.858092e+00,-1.223375e+00,-1.284657e+00,...,-1.584285e+00,-1.327370e+00,-2.674993e+00,-1.785650e+00,-8.517296e-01,-1.902294e+00,-1.028907e+00,-1.483320e+00,-9.704444e-01,-1.130985e+00
25%,-8.172971e-01,-6.199410e-01,-1.009569e+00,-9.521547e-01,-7.475174e-01,-7.082876e-01,-8.366267e-01,-7.124169e-01,-8.439383e-01,-1.158092e+00,...,-8.080919e-01,-1.059949e+00,-7.481341e-01,-7.995909e-01,-8.517296e-01,-7.704298e-01,-1.028907e+00,-7.583028e-01,-9.704444e-01,-1.130985e+00
50%,-1.723959e-01,-3.517037e-02,-6.684748e-02,-2.263875e-01,-7.747876e-02,-1.087651e-01,-3.551376e-02,-9.797159e-02,-9.073497e-02,1.912642e-01,...,-1.481029e-01,1.385177e-01,-6.849133e-03,-1.864599e-01,-3.676439e-01,-1.610397e-01,-1.878280e-01,5.604722e-02,-1.866022e-01,-3.089515e-02
75%,6.449466e-01,5.407618e-01,6.064481e-01,6.444137e-01,6.530419e-01,5.990991e-01,6.332107e-01,5.451257e-01,6.078147e-01,7.152393e-01,...,6.957261e-01,7.120526e-01,1.230724e+00,6.789790e-01,6.468746e-01,5.791401e-01,5.710943e-01,8.897656e-01,6.248011e-01,5.008279e-01
max,3.574847e+00,3.357384e+00,4.076501e+00,3.833117e+00,3.726249e+00,4.129929e+00,3.135303e+00,3.829740e+00,3.777871e+00,3.391707e+00,...,2.013366e+00,2.721112e+00,1.263565e+00,1.848110e+00,3.440715e+00,2.229043e+00,4.055806e+00,1.322011e+00,2.137557e+00,2.806800e+00


In [ ]:
display(normalized_df)

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_1209,-0.777697,-0.345741,0.630467,-0.976416,-0.968139,0.032185,-1.349116,-0.446694,-1.109773,0.779790,...,0.002279,0.282112,1.263565,-1.204263,-0.851730,-1.492485,-1.028907,1.320186,-0.970444,-1.130985
TUC_OBS_1065,0.961072,1.381398,0.486118,1.121150,1.323963,0.922682,1.637681,1.286317,1.491368,0.889878,...,0.877947,0.084538,-1.173485,-0.291113,0.597711,0.204116,1.604964,-0.392794,0.199021,0.786292
TUC_OBS_1828,-1.123444,-2.234426,-1.065474,-0.939199,-1.493535,-1.308674,-1.245930,-1.779760,-1.194965,-1.226250,...,-0.785607,-0.522195,0.163591,1.376564,0.667335,0.956718,-0.064806,0.058620,0.940224,0.550001
TUC_OBS_1401,1.641162,-0.642468,1.919134,1.478415,0.825334,0.743600,0.023806,0.629016,0.772517,0.847682,...,0.176405,1.585850,1.263565,1.848110,1.734868,0.331666,-1.028907,1.322011,2.137557,-1.130985
TUC_OBS_2352,-0.281196,-0.614512,-0.960869,-0.427671,-0.499077,-1.498373,-0.401520,-0.854401,-0.582557,-1.156250,...,0.414210,0.411158,0.557913,1.702104,1.811024,1.783815,1.664777,0.333100,1.579958,1.654897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_1547,0.961543,-0.615275,1.495479,0.833550,0.456695,0.255132,-0.134051,0.018637,0.457668,0.763463,...,-1.212828,0.908994,1.263565,1.848110,1.081910,0.429446,-1.028907,1.322011,2.137557,-1.130985
TUC_OBS_976,0.246911,0.607301,1.017065,0.196469,0.454859,0.367331,0.754573,0.437984,0.494358,0.835700,...,0.515304,-0.229009,-0.481407,-0.286403,-0.184577,-0.002309,0.308515,-0.275321,-0.100148,0.229943
TUC_OBS_1061,0.819341,1.125567,0.602243,0.885540,0.964649,0.812264,1.260800,1.022262,1.065556,0.710470,...,0.877947,0.163610,-0.604453,-0.204075,0.288131,-0.029471,0.760702,-0.177075,0.121275,0.389913


In [ ]:
obj_class

'BH'

In [ ]:
processed_data_all = pd.concat([data_id , normalized_df] , axis=1)
processed_data_all.to_csv('../processed_data/v3/47_tuc_'+obj_class+'_rf_impute.csv')

In [ ]:
processed_data_all = pd.concat([data_id , imp_data] , axis=1)
processed_data_all.to_csv('../processed_data/v3/47_tuc_'+obj_class+'_rf_impute_no_norm.csv')

In [ ]:
processed_data_all

,class,src_n,src_id,significance,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_1209,TUC,CXOGLB J002415.9-720451,TC0122,7.89,-6.995249,-6.667360,-5.673662,-7.282439,-7.145998,-6.187822,...,0.441000,-0.236700,1.000000,-0.620200,-1.000000,-0.712700,-1.000000,0.998100,-0.999400,-0.99940
TUC_OBS_1065,TUC,CXOGLB J002407.8-720441,TC0108,24.52,-5.852633,-5.183626,-5.766750,-5.926648,-5.672028,-5.651306,...,0.684400,-0.330400,-0.113100,-0.135500,-0.356700,0.068100,-0.119300,-0.222400,-0.247300,-0.02620
TUC_OBS_1828,TUC,CXOGLB J002404.4-720432,TC0199,4.23,-7.222454,-8.289872,-6.767348,-7.258384,-7.483861,-6.995679,...,0.222000,-0.618144,0.497597,0.749703,-0.325799,0.414458,-0.677629,0.099233,0.229378,-0.14614
TUC_OBS_1401,TUC,CXOGLB J002353.4-720511,TC0148,5.20,-5.405718,-6.922269,-4.842619,-5.695725,-5.992679,-5.759201,...,0.489400,0.381600,1.000000,1.000000,0.148000,0.126800,-1.000000,0.999400,0.999400,-0.99940
TUC_OBS_2352,TUC,CXOGLB J002434.1-720333,TC0270,4.83,-6.668978,-6.898253,-6.699890,-6.927750,-6.844360,-7.109970,...,0.555500,-0.175500,0.677700,0.922500,0.181800,0.795100,-0.099300,0.294800,0.640800,0.41470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_1547,TUC,CXOGLB J002359.3-720448,TC0168,3.42,-5.852324,-6.898908,-5.115828,-6.112543,-6.229737,-6.053499,...,0.103250,0.060600,1.000000,1.000000,-0.141800,0.171800,-1.000000,0.999400,0.999400,-0.99940
TUC_OBS_976,TUC,CXOGLB J002406.3-720452,TC0101,7.20,-6.321937,-5.848630,-5.424350,-6.524329,-6.230918,-5.985900,...,0.583600,-0.479100,0.203000,-0.133000,-0.703900,-0.026900,-0.552800,-0.138700,-0.439700,-0.30860
TUC_OBS_1061,TUC,CXOGLB J002407.8-720441,TC0108,24.52,-5.945770,-5.403403,-5.691863,-6.078938,-5.903090,-5.717831,...,0.684400,-0.292900,0.146800,-0.089300,-0.494100,-0.039400,-0.401600,-0.068700,-0.297300,-0.22740
